<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# Exporting model data to GIS

This notebook shows how to export model data so it can be viewed in GIS (QGIS or ArcMAP).

In [ ]:
import os
import flopy
import geopandas as gpd
import matplotlib.pyplot as plt
import nlmod
import numpy as np
import xarray as xr
from IPython.display import FileLink, FileLinks
from shapely.geometry import Polygon

In [ ]:
print(f"nlmod version: {nlmod.__version__}")

nlmod.util.get_color_logger("INFO");

## Model types

### structured grid

In [ ]:
model_ws = "ijmuiden"
model_name = "IJmuiden"

In [ ]:
ds_struc = xr.load_dataset(
    os.path.join(model_ws, "cache", f"{model_name}.nc"), mask_and_scale=False
)
ds_struc

In [ ]:
# create gisdir
gisdir_struc = os.path.join(model_ws, "gis")
if not os.path.exists(gisdir_struc):
    os.mkdir(gisdir_struc)

### vertex grid

In [ ]:
model_name = "IJm_planeten"

In [ ]:
ds_vert = xr.load_dataset(
    os.path.join(model_ws, "cache", f"{model_name}.nc"), mask_and_scale=False
)

# get modelgrid
sim = flopy.mf6.MFSimulation.load(
    sim_name="mfsim.nam", sim_ws=ds_vert.model_ws, load_only=["DISV"]
)
gwf = sim.get_model(ds_vert.model_name)

# get vertices
ds_vert["vertices"] = nlmod.grid.get_vertices(ds_vert)
ds_vert

In [ ]:
# create gisdir
gisdir_vert = os.path.join(model_ws, "gis")
if not os.path.exists(gisdir_vert):
    os.mkdir(gisdir_vert)

## Export vector data

### structured

In [ ]:
# write model data to a geopackage
fname_geopackage = nlmod.gis.ds_to_vector_file(ds_struc, gisdir=gisdir_struc)

# get download link
FileLink(fname_geopackage, result_html_prefix="klik hier om te downloaden -> ")

### vertex grid

In [ ]:
# write model data to a geopackage
fname_geopackage = nlmod.gis.ds_to_vector_file(ds_vert, gisdir=gisdir_vert)

## Export griddata

The model data can be exported to a netcdf file that can be visualised in Qgis. For a structured model the standard model dataset (xarray.Dataset) can be exported to a netdf file. For a vertex model you have to convert the model dataset to a certain format before you can write it to a netcdf and read it with Qgis. With the code below we export the vertex model dataset to a netcdf file ('model_qgis.nc') that can be read using Qgis.

### structured grid

In order to load a structured grid netcdf file in Qgis use the "Add Raster Layer.." option.

In [ ]:
# write model data to a netcdf file
fname = os.path.join(gisdir_struc, "model_struc_qgis.nc")
ds_struc.to_netcdf(fname)

# get download link
FileLink(fname, result_html_prefix="klik hier om te downloaden -> ")

### vertex grid

In order to load a vertex netcdf file in Qgis use the "Add Mesh Layer.." option.

In [ ]:
# write model data to a netcdf file
fname = os.path.join(gisdir_vert, "model_vert_qgis.nc")
out = nlmod.gis.ds_to_ugrid_nc_file(ds_vert.drop_vars("rch_name"), fname)

# get download link
FileLink(fname, result_html_prefix="klik hier om te downloaden -> ")

## Add symbology (QGIS)

It is always nice to have automatic symbology for your vector data. Some thoughts:
- QGIS can save symbology of a single shapefile in a .qml file
- In QGIS you can add a .qml file to a geopackage thus saving the symbology to a single file.
- You can create a .qml file in QGIS from existing symbology.
- a .qml file is an xml file so theoretically it is possible to modify a .qml file with Python based on the properties of the data.

Some limitations of the current gis functions:
- when exporting shapefiles to gis, attributes cannot have names longer
than 10 characters. Now the automatic character shortening of fiona is
used. This is not optimal.
- when exporting data variables with dimension time only the mean values
in time are exported in the shapefile to avoid extremely large files.